In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Decision Trees
## Ivan Makaveev, 2MI0600203

In [2]:
file_path = "./breast-cancer.data"
seed = 7000

use_prepruning = True
prepruning_mode = 'G'
use_postpruning = False
postpruning_mode = 'E'

fill_missing = False

In [3]:
col_names = ["class", "age", "menopause", "tumor-size", "inv-nodes", "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]
data = pd.read_csv(file_path, names = col_names)
data['deg-malig'] = data['deg-malig'].apply(str)
data.head()

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [4]:
def fill_missing_with_mode(dataframe, column):
    dataframe[column] = dataframe[column].fillna(dataframe[column].mode()[0])

In [5]:
if(fill_missing):
    data = data.replace("?", np.nan)
    for col in col_names[1:]:
        fill_missing_with_mode(data, col)

In [6]:
def evalute_accuracy(model, test_data, target_attribute):
    correct = 0
    for record in test_data.iloc:
        correct += model.predict(record) == record[target_attribute]
    
    return correct * 100 / len(test_data)

def evaluate_mean(value_list):
    return sum(value_list) / len(value_list)

def evaluate_sd(value_list):
    mean = evaluate_mean(value_list)
    sd = 0
    for val in value_list:
        sd += (val - mean) * (val - mean)
    sd /= len(value_list)
    
    return np.sqrt(sd)

In [7]:
class TreeNode:
    def __init__(self, attribute_id=None, is_leaf=False, result_class=None):
        self.attribute_id = attribute_id
        self.is_leaf = is_leaf
        self.result_class = result_class
        self.children = dict()
    
    def add_child(self, value, child):
        self.children[value] = child

class DecisionTree:
    def __init__(self, max_depth=np.inf, min_examples=1, min_gain=0.0):
        if(max_depth < 1):
            raise ValueError("Cannot have max depth less than 1")

        self.max_depth = max_depth
        self.min_examples = min_examples
        self.min_gain = min_gain
        self.root = None
        self.size = 0
    
    def extract_mode(self, dataframe, attribute, default):
        mode_data = dataframe[attribute].mode()
        if(mode_data.size == 1):
            return mode_data[0]
        else:
            return default
    
    def eval_entropy(self, data_series):
        entropy = 0
        for val in data_series.value_counts(normalize=True):
            entropy += -val*np.log2(val)
        return entropy
    
    def pick_best_attribute(self, dataframe, target_attr, attributes):
        initial_entropy = self.eval_entropy(dataframe[target_attr])
        attr_scores = dict()
        
        for attr in attributes:
            res_gain = initial_entropy
            for val in dataframe[attr].unique():
                filtered_data = dataframe[dataframe[attr] == val].drop(columns=[attr])
                res_gain -= len(filtered_data) / len(dataframe) * self.eval_entropy(filtered_data[target_attr])
            attr_scores[attr] = res_gain
            
        best_attr = max(attr_scores, key=attr_scores.get)
        return best_attr, attr_scores[best_attr]
    
    def train(self, dataframe, target_attribute):
        self.size = 0
        attributes = dataframe.columns.to_list()
        attributes.remove(target_attribute)
        self.root = self.build_node(dataframe, target_attribute, 1, attributes, dataframe[target_attribute].mode()[0])

    def build_node(self, dataframe, target_attr, depth, attributes, default):
        self.size += 1
        if(len(dataframe) == 0):
            return TreeNode(None, True, default)
        elif(dataframe[target_attr].unique().size == 1):
            return TreeNode(None, True, dataframe[target_attr].unique()[0])
        elif(len(attributes) == 0 or len(dataframe) <= self.min_examples or depth > self.max_depth):
            return TreeNode(None, True, self.extract_mode(dataframe, target_attr, default))
        else:
            best_attribute, gain = self.pick_best_attribute(dataframe, target_attr, attributes)
            if(gain < self.min_gain):
                return TreeNode(None, True, self.extract_mode(dataframe, target_attr, default))

            df_mode = self.extract_mode(dataframe, target_attr, default)
            new_node = TreeNode(best_attribute, False, df_mode)
            for value in dataframe[best_attribute].unique():
                filtered_data = dataframe[dataframe[best_attribute] == value].drop(columns=[best_attribute])
                child_node = self.build_node(filtered_data, target_attr, depth+1, [x for x in attributes if x != best_attribute], df_mode)
                new_node.add_child(value, child_node)
            return new_node
    
    def predict(self, record):
        curr_node = self.root
        while not curr_node.is_leaf:
            if curr_node.attribute_id not in record.keys():
                return curr_node.result_class
            
            target_value = record[curr_node.attribute_id]
            if target_value not in curr_node.children.keys():
                return curr_node.result_class
            
            curr_node = curr_node.children[target_value]
        
        return curr_node.result_class
    
    def prune_node(self, node, validation_data, target_attribute, accuracy_before):
        if(node.is_leaf):
            return node
        
        for value, child_node in node.children.items():
            node.children[value] = self.prune_node(child_node, validation_data, target_attribute, accuracy_before)
        
        node.is_leaf = True
        accuracy_after = evalute_accuracy(self, validation_data, target_attribute)
        
        if(accuracy_before > accuracy_after):
            node.is_leaf = False
        else:
            node.children = dict()
            node.attribute_id = None
            
        return node
    
    def prune(self, validation_data, target_attribute):
        if(self.root is None):
            raise ValueError("Cannot prune a tree that has not been trained")
        
        self.root = self.prune_node(self.root, validation_data, target_attribute, evalute_accuracy(self, validation_data, target_attribute))       

In [8]:
def train_test_split(dataframe, test_percentage, target_attribute, seed):
    if (test_percentage < 0 or test_percentage > 100):
        raise ValueError("Test percentage must be between 0 and 100")

    test_frac = test_percentage / 100

    test_parts = list()
    train_parts = list()
    
    for _, group in dataframe.sample(frac=1, random_state=seed).groupby(target_attribute):
        group_test_size = int(len(group) * test_frac)

        test_parts.append(group.iloc[:group_test_size])
        train_parts.append(group.iloc[group_test_size:])

    test_df = pd.concat(test_parts)
    train_df = pd.concat(train_parts)

    return train_df, test_df

In [9]:
def evalute_train_performance(model, train_data, target_attr, folds_count = 10):
    fold_size = len(train_data) // folds_count
    folds = list()
    
    for idx in range(folds_count - 1):
        start_idx = fold_size * idx
        end_idx = fold_size * (idx + 1)
        
        fold_test = train_data.iloc[start_idx:end_idx]
        fold_train = pd.concat([train_data.iloc[:start_idx], train_data.iloc[end_idx:]])
        folds.append((fold_train, fold_test))
    
    
    start_idx = fold_size * (folds_count - 1)
    fold_test = train_data.iloc[start_idx:]
    fold_train = train_data.iloc[:start_idx]
    folds.append((fold_train, fold_test))

    accuracies = list()
    for fold in folds:
        model.train(fold[0], target_attr)
        accuracies.append(evalute_accuracy(model, fold[1], target_attr))
    
    model.train(train_data, target_attr)
    return accuracies, evalute_accuracy(model, train_data, target_attr)

In [10]:
train, test = train_test_split(data, 20, 'class', seed)

In [11]:
dt = DecisionTree()
if(use_prepruning):
    if(prepruning_mode == 'N'):
        dt = DecisionTree(max_depth=3)
    elif(prepruning_mode == 'K'):
        dt = DecisionTree(min_examples=6)
    elif(prepruning_mode == 'G'):
        dt = DecisionTree(min_gain=0.07)

if(use_postpruning and postpruning_mode == 'E'):
    train, pruning_set =  train_test_split(train, 10, 'class', seed)
        
cv_accuracies, train_acc = evalute_train_performance(dt, train, 'class')

print(f"Train set accuracy: {train_acc:.2f}%")
print()

print("10-fold Cross Validation:")
for idx, accuracy in enumerate(cv_accuracies):
    print(f"Fold {idx + 1}: {accuracy:.2f}%")

print(f"Mean: {evaluate_mean(cv_accuracies):.2f}%")
print(f"SD: {evaluate_sd(cv_accuracies):.2f}")

Train set accuracy: 83.84%

10-fold Cross Validation:
Fold 1: 86.36%
Fold 2: 86.36%
Fold 3: 86.36%
Fold 4: 68.18%
Fold 5: 68.18%
Fold 6: 90.91%
Fold 7: 86.36%
Fold 8: 31.82%
Fold 9: 31.82%
Fold 10: 19.35%
Mean: 65.57%
SD: 26.08


In [12]:
if(use_postpruning):
    if(postpruning_mode == 'E'):
        dt.prune(pruning_set, 'class')

In [13]:
test_acc = evalute_accuracy(dt, test, 'class')
print(f"Test set accuracy: {test_acc:.2f}%")

Test set accuracy: 78.95%


# Bonus: Random Forest

In [14]:
class RandomForest:
    def __init__(self, k_trees, max_samples=None, n_features=None, seed=7000):
        if(k_trees < 1):
            raise ValueError("K trees must be at least 1")
        if(max_samples < 1):
            raise ValueError("Max samples must be at least 1")
        if(n_features < 1):
            raise ValueError("N features must be at least 1")
        
        self.k_trees=k_trees
        self.max_samples= max_samples
        self.n_features = n_features
        self.estimators = list()
        self.seed = seed
        
    def bootstrap(self, dataframe, target_attribute, seed):
        n_rows = len(dataframe)
        max_samples = self.max_samples if self.max_samples is not None else n_rows
        if max_samples > n_rows:
            max_samples = n_rows
        
        feature_cols = [c for c in dataframe.columns if c != target_attribute]
        n_features = self.n_features if self.n_features is not None else len(feature_cols)
     
        if n_features > len(feature_cols):
            n_features = len(feature_cols)

        chosen_features = list(np.random.default_rng(seed).choice(feature_cols, size=n_features, replace=False))
        cols = chosen_features + [target_attribute]

        bootstrapped = dataframe[cols].sample(n=max_samples, replace=True, random_state=seed)
        return bootstrapped
    
    def train(self, dataframe, target_attribute):
        local_seed = self.seed
        for i in range(0, self.k_trees):
            tree = DecisionTree(max_depth=3)
            train_data = self.bootstrap(dataframe, target_attribute, local_seed)
            local_seed += 1
            tree.train(train_data, target_attribute)
            self.estimators.append(tree)
            
    def predict(self, record):
        results = dict()
        for tree in self.estimators:
            curr_res = tree.predict(record)
            if(curr_res not in results.keys()):
                results[curr_res] = 0
            
            results[curr_res] += 1
        
        return max(results, key=results.get)

In [15]:
rf = RandomForest(200, 70, 6, seed)

rf.train(train, 'class')
train_acc = evalute_accuracy(rf, train, 'class')
print(f"Train set accuracy: {train_acc:.2f}%")

Train set accuracy: 81.66%


In [16]:
test_acc = evalute_accuracy(rf, test, 'class')
print(f"Test set accuracy: {test_acc:.2f}%")

Test set accuracy: 73.68%
